In [ ]:
import requests
import json
import pandas as pd
from time import sleep

In [ ]:
df = pd.read_csv('../output/codici_li.csv', dtype=str)
# Filtering for only Comuni
df_comuni = df[df['tipo'] == 'CM']

In [ ]:
# Create a copy of the DataFrame to avoid the warning
df_comuni = df_comuni.copy()

BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241027/TE/07/RE/'

df_comuni['url_aff'] = df_comuni.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_comuni['url'] = df_comuni.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)

In [ ]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [ ]:
comuni_data = []

for idx, url in enumerate(df_comuni['url_aff']):
  response = requests.get(url, headers=headers)
  data = response.json()

  comune = df_comuni.iloc[idx]['desc']
  provincia = df_comuni.iloc[idx]['provincia']
  regione = df_comuni.iloc[idx]['regione']
  url = df_comuni.iloc[idx]['url']

  # Initialize comune_info with basic location details
  comune_info = {
      'comune': comune,
      'provincia': provincia,
      'regione': regione,
      'url': url
  }
  
  # Track unique parties and candidate data
  if 'cand' in data:
      # First, collect party information
      for cand in data['cand']:
          if 'liste' in cand:
              for lista in cand['liste']:
                  party_name = lista['desc_lis_c']
                  comune_info[party_name] = lista['voti']
                  comune_info[f"{party_name}_perc"] = lista['perc']
      
      # Then, add candidate information
      for cand in data['cand']:
          nome_completo = f"{cand['nome']} {cand['cogn']}"
          comune_info[f"{nome_completo}_total_votes"] = cand['voti']
          comune_info[f"{nome_completo}_total_perc"] = cand['perc']

  comuni_data.append(comune_info)
  
  sleep(0.1)

# Create DataFrame
df_data = pd.DataFrame(comuni_data)

In [ ]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

In [ ]:
df_data.to_csv('../output/risultati_LI.csv', index=False, encoding='UTF-8-sig')

### Formatting for Datawrapper visualization

In [ ]:
candidates = df_data[['comune', 'provincia', 'regione', 'MARCO BUCCI_total_votes', 'MARCO BUCCI_total_perc', 'ANDREA ORLANDO_total_votes', 'ANDREA ORLANDO_total_perc']]

candidates.head(2)

In [ ]:
# Basic cleaning for Datawrapper integration
candidates['comune'] = candidates['comune'].str.replace("Ricco'", "Riccò", regex=False)
candidates['comune'] = candidates['comune'].str.replace("Prela'", "Prelà", regex=False)

In [ ]:
# Calculate the left-right index
candidates['left_right_index'] = (candidates['MARCO BUCCI_total_votes'] - candidates['ANDREA ORLANDO_total_votes']) / (candidates['MARCO BUCCI_total_votes'] + candidates['ANDREA ORLANDO_total_votes'])
candidates.sample(5)

In [ ]:
candidates.to_csv('../output/viz/ris_candidati_LI.csv', index=False, encoding='UTF-8-sig')